# **DeepLandforms**

Author: giacomo.nodjoumi@hyranet.info - g.nodjoumi@jacobs-university.de

## DeepLandforms Segmentation

With this notebook, users can use custom trained models for instance segmentation models on custom dataset of georeferenced images.
The output consist of a folder containing:
* Source Images in which at least one detection occurred
* Label file in COCO json format for each image
* Geopackage containing a single layer with image name, confidence leve, class.

## Usage

* Put or link the dataset into the **DeepLandforms** *.env* file
* Run docker-compose up
* Edit the *configs* section by editing the following parameters:
------------------------------------------------------------------
| **Parameter** | **Function** | **Common Values** |
| ---- | ---- | ---- |
| **batch_size** | N° of images to be processed at once | Depending on VRAM and image size, up to 8 per 8GB VRAM |
| **geopackage_name** |  Name of the final geopackage |  |
| **proj_geopackage_name** | Name of the final geopackage in custom projection | |
| **model_path** | local path and name of the model  | it should start with /pre-trained_models/ |
| **model_yaml** | Model Architecture | MASK-R-CNN in this work | EDIT according to trained model selected |
| **dst_crs** | CRS of the geopackage | provide as WKT or proj4 |

------------------------------------------------------------------
Then just execute the notebook and monitor the training in **Tensorboard** container.

## Funding
*This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.*

In [1]:
import cv2
from datetime import datetime
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.catalog import Metadata
import numpy as np
import os
import geopandas as gpd
import pandas as pd
import psutil
from pyproj import CRS
#from pycocotools import mask
import random
import rasterio as rio
from rasterio.plot import reshape_as_image
import shutil
import torch
from tqdm import tqdm
from utils.GenUtils import get_paths
from utils.detectron_utils import CustomPredictor
from utils.geoshape_utils import parallel_funcs, chunk_creator, mask2shape, pred2coco, pred2shape, crs_validator

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
torch.cuda.is_available()
torch.cuda.get_device_name()

1.12.1+cu116


'NVIDIA GeForce RTX 2070 with Max-Q Design'

## CONFIGURATION - edit befor run

In [4]:
batch_size = 1
image_path = '../data/BC_n_SQCRP_n_CellSize_10_m__LIM_n_None_px_cog_n/test_data'
geopackage_name = '/Inferred_Shapes.gpkg' ## Example for HiRISE 
proj_geopackage_name = '/Inferred_Shapes_projected.gpkg' ## Example for HiRISE 
model_path = '/home/user/data/TrainedModels/5m_re-ok-2/model_final.pth'
model_yaml = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" ## EDIT according to trained model selected
#src_crs = CRS.from_user_input('PROJCRS["Equirectangular MARS", BASEGEOGCRS["GCS_MARS",DATUM["unnamed",ELLIPSOID["unnamed",3393833.2607584,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Reference meridian",0,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]]],CONVERSION["unnamed",METHOD["Equidistant Cylindrical",ID["EPSG",1028]],PARAMETER["Latitude of natural origin",20,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",180,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Latitude of 1st standard parallel",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]')
#dst_crs = CRS.from_user_input('PROJCS["Equirectangular MARS",GEOGCS["GCS_MARS",DATUM["unnamed",SPHEROID["unnamed",3395582.0270805,0]],PRIMEM["Reference meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",10],PARAMETER["central_meridian",180],PARAMETER["standard_parallel_1",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')
#dst_crs = CRS.from_wkt('PROJCRS["Equirectangular MARS", BASEGEOGCRS["GCS_MARS", DATUM["unnamed", ELLIPSOID["unnamed",3396190,0, LENGTHUNIT["metre",1,  ID["EPSG",9001]]]], PRIMEM["Reference meridian",0, ANGLEUNIT["degree",0.0174532925199433,		ID["EPSG",9122]]]], CONVERSION["Equidistant Cylindrical", METHOD["Equidistant Cylindrical", ID["EPSG",1028]], PARAMETER["Latitude of 1st standard parallel",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8823]], PARAMETER["Longitude of natural origin",180, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8802]], PARAMETER["False easting",0, LENGTHUNIT["metre",1], ID["EPSG",8806]], PARAMETER["False northing",0, LENGTHUNIT["metre",1], ID["EPSG",8807]]], CS[Cartesian,2], AXIS["easting",east, ORDER[1], LENGTHUNIT["metre",1, ID["EPSG",9001]]], AXIS["northing",north, ORDER[2], LENGTHUNIT["metre",1, ID["EPSG",9001]]]]')
dst_crs_2 = CRS.from_user_input('GEOGCRS["GCS_Mars_2000_Sphere", DATUM["Mars_2000_(Sphere)", ELLIPSOID["Mars_2000_Sphere_IAU_IAG",3396190,0, LENGTHUNIT["metre",1]], ID["ESRI",106971]], PRIMEM["Reference_Meridian",0, ANGLEUNIT["Degree",0.0174532925199433]], CS[ellipsoidal,2], AXIS["longitude",east, ORDER[1], ANGLEUNIT["Degree",0.0174532925199433]], AXIS["latitude",north, ORDER[2], ANGLEUNIT["Degree",0.0174532925199433]]]')
#dst_crs = CRS.from_wkt('GEOGCS["Moon 2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')

In [5]:
out_dir = image_path+'/outputs'
os.makedirs(out_dir, exist_ok=True)
class_file = os.path.dirname(model_path)+'/trained_classes.csv'
class_df = pd.read_csv(class_file)
classes = class_df[class_df.columns[0]].tolist()
meta = Metadata()
meta.set(thing_classes=classes)

namespace(thing_classes=['Type-4', 'Crater', 'Type-3', 'Type-1', 'Type-2'])

In [6]:
images = get_paths(image_path,'tiff')
src_crs = CRS.from_wkt(rio.open(image_path+'/'+images[0]).crs.to_wkt())

In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_yaml))
cfg.MODEL.ROI_HEADS.NUM_CLASSES =  len(classes)
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
#cfg.INPUT.MIN_SIZE_TEST = 32
#cfg.INPUT.MAX_SIZE_TEST = 32
#cfg.INPUT.MIN_SIZE_TRAIN =32
#cfg.INPUT.MAX_SIZE_TRAIN = 32
cfg.SOLVER.IMS_PER_BATCH = batch_size
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE =  10000
cfg.SOLVER.AMP.ENABLED=True
#cfg.MODEL.RPN.IN_FEATURES = ['p2', 'p3', 'p4', 'p5', 'p6']
#cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0, 4.0, 8.0]]
#cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4], [8], [16], [64], [128]]


In [9]:
cols = ['Name','Class','Score']
dst_gpkg = out_dir+'/Inferred_Shapes_projected.gpkg'
proc_csv = out_dir+'/Processed.csv'
#try:
#    geoshapes = gpd.read_file(dst_gpkg)
#except Exception as e:
#    print(e)
#    geoshapes = None
    #geoshapes = gpd.GeoDataFrame(columns=cols)
#    pass
try:
    proc_df = pd.read_csv(proc_csv)
except Exception as e:
    print(e)
    proc_df = pd.DataFrame(columns=['Name','Detections'])
    pass

[Errno 2] No such file or directory: '../data/BC_n_SQCRP_n_CellSize_10_m__LIM_n_None_px_cog_n/test_data/outputs/Processed.csv'


In [10]:
chunks = list(chunk_creator(images,batch_size))
len(chunks)

19

In [11]:
def geopackager(gdf,i,geoshapes):
    try:
        geoshapes.crs
    except:
        geoshapes = gdf
    init_crs = geoshapes.crs
    try:
        if gdf.crs != init_crs:
            gdf = gdf.to_crs(init_crs)
        #geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,gdf],ignore_index=True),crs=init_crs)
    except Exception as e:
        #geoshapes = gdf
        #geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,gdf],ignore_index=True),crs=init_crs)
        print(e)            
        pass
    geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,gdf]).drop_duplicates().reset_index(drop=True),crs=init_crs)
    geoshapes = geoshapes[np.isnan(geoshapes['geometry'].area) == False]
    geoshapes.to_file(out_dir+proj_geopackage_name, driver='GPKG', crs=init_crs) 
    return geoshapes

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #### To be removed

In [14]:
#CP = CustomPredictor(cfg)
from detectron2.engine import DefaultPredictor
CP = DefaultPredictor(cfg)
JOBS=psutil.cpu_count(logical=False)
with tqdm(total=len(images),
             desc = 'Generating Images',
             unit='File') as pbar:
    start = datetime.now()
    gdf_list = []
    for d in range(len(chunks)):
    #for d in range(1):
        chunk = list(chunks[d])

        lambda_f = lambda element:element not in proc_df['Name'].to_list()
        #lambda_f = lambda element:element not in geoshapes['Name'].to_list()
        filtered = filter(lambda_f, chunk)

        chunk = list(filtered)
        if len(chunk)>0:
            paths = [image_path+'/'+ele for ele in chunk]
            open_images = [rio.open(img_path) for img_path in paths]
            #imgs = [reshape_as_image(image.read()) for image in open_images]
            imgs, windows_dict = windowed_imgs(open_images[0], 0, 250, paths[0])
            for i in range(len(imgs)):
                #print(windows_dict['Transforms'][i])
                               
                predictions = CP(imgs[i])
                masks = predictions['instances'].pred_masks.cpu().numpy()           
                if len(masks)>0:                
                    for l in range(len(predictions)):
                        print(l)
                        #gdf = pred2shape(predictions, windows_dict['Names'][i], open_images[i],classes, JOBS, out_dir, i)    
                        gdf = pred2shape(predictions, l,windows_dict['Names'][i], windows_dict['Transforms'][i], open_images[0].crs ,classes, JOBS, out_dir, i)    
                        gdf_list.append(gdf)
                        tmp_df = pd.DataFrame([[os.path.basename(windows_dict['Names'][i]),len(gdf)]],columns=["Name","Detections"])
                        proc_df = pd.concat([proc_df,tmp_df],ignore_index=True)
                        proc_df.to_csv(proc_csv, index=False)                    #tmp_df['Detections']=len(geoshapes)
                        try:
                            geoshapes = gpd.read_file(dst_gpkg)                        
                        except Exception as e:
                            print(e)
                            geoshapes = None
                        geoshapes = geopackager(gdf, i, geoshapes)     

                #else:
                #    for i in range(len(paths)):
                #        tmp_df = pd.DataFrame([[os.path.basename(paths[i]),0]],columns=["Name","Detections"])
                #        proc_df = pd.concat([proc_df,tmp_df],ignore_index=True)
                #        #proc_df = proc_df.append(tmp_df,ignore_index=True)
                #        proc_df.to_csv(proc_csv, index=False)

            pbar.update(batch_size)        
            stop = datetime.now()
        print(stop-start)

Generating Images:   0%|          | 0/19 [00:00<?, ?File/s]


NameError: name 'windowed_imgs' is not defined

In [ ]:
windows_dict

In [ ]:
imgs[0]

In [ ]:
masks = predictions['instances'].pred_masks.cpu().numpy()
len(masks)

In [ ]:
len(predictions)

In [ ]:
#CP = CustomPredictor(cfg)
from detectron2.engine import DefaultPredictor
CP = DefaultPredictor(cfg)
JOBS=psutil.cpu_count(logical=False)
with tqdm(total=len(images),
             desc = 'Generating Images',
             unit='File') as pbar:
    start = datetime.now()
    gdf_list = []
    for d in range(len(chunks)):
    #for d in range(1):
        chunk = list(chunks[d])

        lambda_f = lambda element:element not in proc_df['Name'].to_list()
        #lambda_f = lambda element:element not in geoshapes['Name'].to_list()
        filtered = filter(lambda_f, chunk)

        chunk = list(filtered)


In [ ]:
d = 0
chunk = list(chunks[d])
chunk

In [ ]:
paths = [image_path+'/'+ele for ele in chunk]
            

In [ ]:
paths

In [ ]:
open_images = [rio.open(img_path) for img_path in paths]
#imgs = [reshape_as_image(image.read()) for image in open_images]

In [ ]:
overlap = 0

In [ ]:
max_dim = 32

In [ ]:
savename = chunk[0]
oxt = 'tiff'
from rasterio.windows import Window
from rasterio.enums import Resampling
imgs = []

In [ ]:
len(imgs)

In [ ]:
imgs[0].shape

In [ ]:
for i in range(len(imgs)):
    predictions = CP(imgs[i])
    masks = predictions['instances'].pred_masks.cpu().numpy()           
    if len(masks)>0:                
        for i in range(len(predictions)):
            gdf = pred2shape(predictions, paths[i], open_images[i],classes, JOBS, out_dir, i)    
            gdf_list.append(gdf)
            tmp_df = pd.DataFrame([[os.path.basename(paths[i]),len(gdf)]],columns=["Name","Detections"])
            proc_df = pd.concat([proc_df,tmp_df],ignore_index=True)
            proc_df.to_csv(proc_csv, index=False)                    #tmp_df['Detections']=len(geoshapes)
            try:
                geoshapes = gpd.read_file(dst_gpkg)                        
            except Exception as e:
                print(e)
                geoshapes = None
            geoshapes = geopackager(gdf, i, geoshapes)     

In [ ]:
dst_crs = open_images[0].crs

In [ ]:
if len(masks)>0:                
    for i in range(len(predictions)):
        gdf = pred2shape(predictions, paths[i], open_images[i],classes, JOBS, out_dir, i)    
        gdf_list.append(gdf)
        tmp_df = pd.DataFrame([[os.path.basename(paths[i]),len(gdf)]],columns=["Name","Detections"])
        proc_df = pd.concat([proc_df,tmp_df],ignore_index=True)
        proc_df.to_csv(proc_csv, index=False)                    #tmp_df['Detections']=len(geoshapes)
        try:
            geoshapes = gpd.read_file(dst_gpkg)                        
        except Exception as e:
            print(e)
            geoshapes = None
        geoshapes = geopackager(gdf, i, geoshapes)     

In [ ]:
geoshapes.crs